In [1]:
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import GPflow
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
get_ipython().magic('matplotlib inline')

#Obtener los archivos de datos.
DIR = 'database/'
files=([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
n=len(files)
D=np.array([])
for i in range(0,len(files)):    
    source=DIR+files[i]
    file = open(source, "r")
    d=[]
    f=[]
    for line in file: 
        if len(line.split())>2:
            f.append(line.split()[2])        
        if len(line.split())==9:
            d.append(line.split()[7])
        elif len(line.split())==5:
            d.append(line.split()[3])
        else:
            d.append(0)
    d.pop(0)
    myarray = np.asarray(d)
    file.close()    
    D = np.append(D, np.transpose(myarray), axis=0)

D=D.reshape(n,int(D.shape[0]/n))
D=D.astype(np.float)

#Borar datos defectuosos/no adquiridos.
ind=[]
for j in range(0,len(D)):
    for i in range(0,len(D[0])):
        if D[j,i]==0:
            ind.append(i)
a=list(set(ind))
a=np.asarray(a)
Fech=np.asarray(f)
Fech=np.delete(Fech,ind)
Fech=np.arange(len(Fech))
RD=np.delete(D[0],ind)
PH=np.delete(D[1],ind)
DO=np.delete(D[2],ind)
EC=np.delete(D[3],ind)
T=np.delete(D[4],ind)

#Separa los conjuntos en Entrenamiento y Test.
PE=.75
PT=.25
n=len(DO)
DO_train=DO[0:np.round(n*PE).astype(int)]
DO_train.shape=(len(DO_train),1)
DO_test=DO[np.round(n*PE).astype(int)+1:-1]
DO_test.shape=(len(DO_test),1)


EC_train=EC[0:np.round(n*PE).astype(int)]
EC_train.shape=(len(EC_train),1)
EC_test=EC[np.round(n*PE).astype(int)+1:-1]
EC_test.shape=(len(EC_test),1)


RD_train=RD[0:np.round(n*PE).astype(int)]
RD_train.shape=(len(RD_train),1)
RD_test=RD[np.round(n*PE).astype(int)+1:-1]
RD_test.shape=(len(RD_test),1)


T_train=T[0:np.round(n*PE).astype(int)]
T_train.shape=(len(T_train),1)
T_test=T[np.round(n*PE).astype(int)+1:-1]
T_test.shape=(len(T_test),1)


PH_train=PH[0:np.round(n*PE).astype(int)]
PH_train.shape=(len(PH_train),1)
PH_test=PH[np.round(n*PE).astype(int)+1:-1]
PH_test.shape=(len(PH_test),1)

Fech_train=Fech[0:np.round(n*PE).astype(int)]
Fech_train=np.divide(Fech_train,float(len(Fech_train)))
Fech_test=Fech[np.round(n*PE).astype(int)+1:-1]
Fech_test=np.divide(Fech_test,float(len(Fech_train)))

#Graficar los valores
p=range(0,len(DO))

plt.plot(p[0:len(T_train)],T_train,'b')
plt.plot(p[len(T_train)+1:-1],T_test,'r')
#lt.show()

#Métricas a utilizar.
def RMSE(obs,med):
    r=np.power(np.subtract(obs,med),2)
    return np.sqrt((1/len(obs)*np.sum(r)))

def NS(obs,med):
    mean=np.mean(obs)
    num=np.sum(np.power(np.subtract(obs,med),2))
    den=np.sum(np.power(np.subtract(obs,np.mean(obs)),2))
    return 1-num/den

def MARE(obs,med):
    num=np.subtract(obs,med)
    return (1/len(obs))*100*np.sum(np.absolute(np.divide(num,med)))

def R(obs,med):
    num=np.sum(np.multiply(np.subtract(obs,np.mean(obs)),np.subtract(med,np.mean(med))))
    den=np.multiply(np.sum(np.power(np.subtract(obs,np.mean(obs)),2)),np.sum(np.power(np.subtract(med,np.mean(med)),2)))
    return num/np.sqrt(den)

def Norm(x,X):
    return np.divide(np.subtract(x,np.mean(X)),np.sqrt(np.var(X)))
def DeNorm(x,X):
    return np.multiply(np.add(x,np.mean(X)),np.sqrt(np.var(X)))

Fech_train.shape=(Fech_train.shape[0],1)
Fech_test.shape=(Fech_test.shape[0],1)

ImportError: No module named GPflow

In [ ]:
# GP-KSE para el DO
k = GPflow.kernels.RBF(input_dim=1)
m = GPflow.gpr.GPR(Fech_train, DO_train, kern=k)
m
m.optimize()
m
mean, var = m.predict_y(Fech_test)
plt.plot(Fech_test,mean,'b')
plt.plot(Fech_test,DO_test,'r')
inf=mean - 2*np.sqrt(var)
sup=mean + 2*np.sqrt(var)
plt.fill_between(Fech_test[:,0], inf[:,0], sup[:,0],color='blue', alpha=0.2)
plt.show()

In [ ]:
# GP-Cosine para el DO
k = GPflow.kernels.Cosine(input_dim=1)
m = GPflow.gpr.GPR(Fech_train, DO_train, kern=k)
m
m.optimize()
m
mean, var = m.predict_y(Fech_test)
plt.plot(Fech_test,mean,'b')
plt.plot(Fech_test,DO_test,'r')
inf=mean - 2*np.sqrt(var)
sup=mean + 2*np.sqrt(var)
plt.fill_between(Fech_test[:,0], inf[:,0], sup[:,0],color='blue', alpha=0.2)
plt.show()

In [ ]:
# GP-Periodic Kernel para el DO
k1 =GPflow.kernels.PeriodicKernel(input_dim=1)
k2 =GPflow.kernels.PeriodicKernel(input_dim=1)
k=k1+k2
m = GPflow.gpr.GPR(Fech_train, DO_train, kern=k)
m
m.optimize()
m
print(m)
mean, var = m.predict_y(Fech_test)
plt.plot(Fech_test,mean,'b')
plt.plot(Fech_test,DO_test,'r')
inf=mean - 2*np.sqrt(var)
sup=mean + 2*np.sqrt(var)
plt.fill_between(Fech_test[:,0], inf[:,0], sup[:,0],color='blue', alpha=0.2)
plt.show()

In [ ]:
# GP-Periodic Kernel para el DO
k1 =GPflow.kernels.PeriodicKernel(input_dim=1)
k2 =GPflow.kernels.PeriodicKernel(input_dim=1)
k3 =GPflow.kernels.PeriodicKernel(input_dim=1)
k=k1+k2+k3
m = GPflow.gpr.GPR(Fech_train, DO_train, kern=k)
print(m)
m.kern.periodickernel_1.period.fixed = True
m.kern.periodickernel_1.period=0.61
m
m.optimize()
m
print(m)
mean, var = m.predict_y(Fech_test)
plt.plot(Fech_test,mean,'b')
plt.plot(Fech_test,DO_test,'r')
inf=mean - 2*np.sqrt(var)
sup=mean + 2*np.sqrt(var)
plt.fill_between(Fech_test[:,0], inf[:,0], sup[:,0],color='blue', alpha=0.2)
plt.show()

In [ ]:
# GP-Locally Periodic Kernel para el DO
k =(GPflow.kernels.PeriodicKernel(input_dim=1)+GPflow.kernels.PeriodicKernel(input_dim=1))*GPflow.kernels.RBF(input_dim=1)
m = GPflow.gpr.GPR(Fech_train, DO_train, kern=k)
m
m.optimize()
m
mean, var = m.predict_y(Fech_test)
plt.plot(Fech_test,mean,'b')
plt.plot(Fech_test,DO_test,'r')
inf=mean - 2*np.sqrt(var)
sup=mean + 2*np.sqrt(var)
plt.fill_between(Fech_test[:,0], inf[:,0], sup[:,0],color='blue', alpha=0.2)
plt.show()